# Importing the required libraries


In [ ]:
import cv2
import numpy as np
import dlib                       # For detecting face in the camera and puting facial landmarks on it 
from imutils import face_utils

 # Grabing the video camera input in a variable 

In [6]:
cap=cv2.VideoCapture(0)         

# Detecting face in the image and puting 68 facial landmarks on it

In [7]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

 # Algorithm to calculate the drowsiness by computing the gap between the eye landmarks

In [8]:
sleep=0
drowsy=0
active=0
status="hey"
color=(0,0,0)

def empty(a):
    pass


def compute(ptA,ptB):
    dist = np.linalg.norm(ptA-ptB)
    return dist


def blinked(a,b,c,d,e,f):
    up=compute(b,d)+compute(c,e)
    down=compute(a,f)
    ratio=up/(2.0*down)
    
    if(ratio>0.25):
        return 2
    elif(ratio>0.21 and ratio<=0.25):
        return 1
    else:
        return 0


# Calculating the drowsiness in the real time

In [5]:

while True:
    _,frame =cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces= detector(gray)

    for face in faces:
        x1=face.left()
        y1=face.top()
        x2=face.right()
        y2=face.bottom()
        
        face_frame=frame.copy()
        cv2.rectangle(face_frame, (x1,y1),(x2,y2),(0,255,0),2)
        
        landmarks= predictor(gray, face)
        landmarks= face_utils.shape_to_np(landmarks)
        
        left_blink= blinked(landmarks[36],landmarks[37],
                            landmarks[38],landmarks[41],
                            landmarks[40],landmarks[39])
        right_blink= blinked(landmarks[42],landmarks[43],
                            landmarks[44],landmarks[47],
                            landmarks[46],landmarks[45])

        if(left_blink==0 or right_blink==0):
            sleep+=1
            drowsy=0
            active=0
            if(sleep>8):
                status="SLEEPING OMG!!!"
                color=(255,0,0)
            
        elif(left_blink==1 or right_blink==1):
            sleep=0
            drowsy+=1
            active=0
            if(drowsy>8):
                status="Drowsy !"
                color=(0,0,255)

        else:
            drowsy=0
            sleep=0
            active+=1
            if(active>8):
                status="Active boi"
                color=(0,255,0)

        cv2.putText(frame,status,(100,100), cv2.FONT_HERSHEY_SIMPLEX,1.2,color,3)                

        for n in range(0,68):
            (x,y)=landmarks[n]
            cv2.circle(face_frame,(x,y),1,(255,255,255),-1)
            
    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    if cv2.waitKey(4) & 0xFF==ord('q'):
        break
                  

# Releasing the camera and closing all the windows

In [ ]:
cv2.destroyAllWindows()
cap=cv2.VideoCapture()
  